# Covid19_India Analysis

In [2]:
import json
from datetime import timedelta
from urllib.request import urlopen

In [3]:
import numpy as np
import pandas as pd


In [11]:
# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium

In [4]:
# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [5]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [6]:
# importing datasets
df = pd.read_csv('complete.csv', parse_dates=['Date'])
df['Name of State / UT'] = df['Name of State / UT'].str.replace('Union Territory of ', '')
df.head()

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Latitude,Longitude,Death,Total Confirmed cases
0,2020-01-30,Kerala,1,0,0,10.8505,76.2711,0,1
1,2020-01-31,Kerala,1,0,0,10.8505,76.2711,0,1
2,2020-02-01,Kerala,2,0,0,10.8505,76.2711,0,2
3,2020-02-02,Kerala,3,0,0,10.8505,76.2711,0,3
4,2020-02-03,Kerala,3,0,0,10.8505,76.2711,0,3


In [7]:
df = df[['Date', 'Name of State / UT', 'Latitude', 'Longitude', 'Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']]
df.columns = ['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths', 'Cured']

for i in ['Confirmed', 'Deaths', 'Cured']:
    df[i] = df[i].astype('int')
    
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Cured']
df['Mortality rate'] = df['Deaths']/df['Confirmed']
df['Recovery rate'] = df['Cured']/df['Confirmed']

df = df[['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Active', 'Deaths', 'Mortality rate', 'Cured', 'Recovery rate']]

df.head()

,Date,State/UT,Latitude,Longitude,Confirmed,Active,Deaths,Mortality rate,Cured,Recovery rate
0,2020-01-30,Kerala,10.8505,76.2711,1,1,0,0.0,0,0.0
1,2020-01-31,Kerala,10.8505,76.2711,1,1,0,0.0,0,0.0
2,2020-02-01,Kerala,10.8505,76.2711,2,2,0,0.0,0,0.0
3,2020-02-02,Kerala,10.8505,76.2711,3,3,0,0.0,0,0.0
4,2020-02-03,Kerala,10.8505,76.2711,3,3,0,0.0,0,0.0


In [8]:
latest = df[df['Date']==max(df['Date'])]

# days
latest_day = max(df['Date'])
day_before = latest_day - timedelta(days = 1)

# state and total cases 
latest_day_df = df[df['Date']==latest_day].set_index('State/UT')
day_before_df = df[df['Date']==day_before].set_index('State/UT')

temp = pd.merge(left = latest_day_df, right = day_before_df, on='State/UT', suffixes=('_lat', '_bfr'), how='outer')
latest_day_df['New cases'] = temp['Confirmed_lat'] - temp['Confirmed_bfr']
latest = latest_day_df.reset_index()
latest.fillna(1, inplace=True)

In [9]:
temp = latest[['State/UT', 'Confirmed', 'Active', 'New cases', 'Deaths', 'Mortality rate', 'Cured', 'Recovery rate']]
temp = temp.sort_values('Confirmed', ascending=False).reset_index(drop=True)

temp.style\
    .background_gradient(cmap="Blues", subset=['Confirmed', 'Active', 'New cases'])\
    .background_gradient(cmap="Greens", subset=['Cured', 'Recovery rate'])\
    .background_gradient(cmap="Reds", subset=['Deaths', 'Mortality rate'])

,State/UT,Confirmed,Active,New cases,Deaths,Mortality rate,Cured,Recovery rate
0,Maharashtra,335,277,0.000000,16,0.047761,42,0.125373
1,Tamil Nadu,309,302,75.000000,1,0.003236,6,0.019417
2,Kerala,286,257,21.000000,2,0.006993,27,0.094406
3,Delhi,219,207,67.000000,4,0.018265,8,0.036530
4,Rajasthan,133,130,25.000000,0,0.000000,3,0.022556
5,Andhra Pradesh,132,130,46.000000,1,0.007576,1,0.007576
6,Karnataka,124,111,14.000000,3,0.024194,10,0.080645
7,Uttar Pradesh,113,97,0.000000,2,0.017699,14,0.123894
8,Telengana,107,103,11.000000,3,0.028037,1,0.009346
9,Madhya Pradesh,99,93,0.000000,6,0.060606,0,0.000000


In [12]:
m = folium.Map(location=[20.5937, 78.9629], tiles='cartodbpositron',
               min_zoom=4, max_zoom=6, zoom_start=4)

for i in range(0, len(latest)):
    if latest.iloc[i]['Confirmed']>0:
        folium.Circle(
            location=[latest.iloc[i]['Latitude'], latest.iloc[i]['Longitude']],
            color='#e84545', 
            fill='#e84545',
            tooltip =   '<li><bold>Name of State / UT : '+str(latest.iloc[i]['State/UT'])+
                        '<li><bold>Confirmed cases  : '+str(latest.iloc[i]['Confirmed'])+
                        '<li><bold>Cured cases  : '+str(latest.iloc[i]['Cured'])+
                        '<li><bold>Death cases  : '+str(latest.iloc[i]['Deaths']),
            radius=int(latest.iloc[i]['Confirmed'])*300).add_to(m)
m

In [13]:
m.save('index.html')

In [15]:
temp = latest.sort_values('Confirmed', ascending=False)
import plotly.graph_objects as go
states=temp['State/UT']

fig = go.Figure(data=[
    go.Bar(name='Confirmed', x=states, y=temp['Confirmed']),
    go.Bar(name='Recovered', x=states, y=temp['Cured']),
    go.Bar(name='Deaths', x=states, y=temp['Deaths'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [16]:
plot_div = plot(fig,output_type='div',include_plotlyjs=False,show_link=False,link_text="",image_width=5000,config={"displaylogo":False})

In [17]:
plot_div

'<div>\n        \n        \n            <div id="dcfad341-a8fb-4d43-bde1-dc6043eef544" class="plotly-graph-div" style="height:100%; width:100%;"></div>\n            <script type="text/javascript">\n                \n                    window.PLOTLYENV=window.PLOTLYENV || {};\n                    \n                if (document.getElementById("dcfad341-a8fb-4d43-bde1-dc6043eef544")) {\n                    Plotly.newPlot(\n                        \'dcfad341-a8fb-4d43-bde1-dc6043eef544\',\n                        [{"name": "Confirmed", "type": "bar", "x": ["Maharashtra", "Tamil Nadu", "Kerala", "Delhi", "Rajasthan", "Andhra Pradesh", "Karnataka", "Uttar Pradesh", "Telengana", "Madhya Pradesh", "Gujarat", "Jammu and Kashmir", "West Bengal", "Punjab", "Haryana", "Bihar", "Chandigarh", "Assam", "Ladakh", "Andaman and Nicobar Islands", "Uttarakhand", "Chhattisgarh", "Goa", "Himachal Pradesh", "Odisha", "Puducherry", "Manipur", "Jharkhand", "Mizoram", "Arunachal Pradesh"], "y": [335, 309, 286,

In [12]:
temp = latest.sort_values('Confirmed', ascending=False)
state_order = temp['State/UT']
temp['']
fig = px.bar(temp, 
             x="Confirmed", y="State/UT", color='State/UT', title='Confirmed',
             orientation='h', text='Confirmed', height=len(temp)*35,
             color_discrete_sequence = px.colors.qualitative.Vivid)
fig.show()

In [14]:
temp = latest[latest['Deaths']>0].sort_values('Deaths')
fig_d = px.bar(temp, y="State/UT", x="Deaths", orientation='h', title='Deaths', color_discrete_sequence = ['#ff2e63'], text='Deaths', height=len(temp)*40)

temp = latest[latest['Cured']>0].sort_values('Cured')
fig_r = px.bar(temp, y="State/UT", x="Cured", orientation='h', title='Cured', color_discrete_sequence = ['#2c786c'], text='Cured', height=len(temp)*40)

fig = make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.2, subplot_titles=("Deaths", "Recovered"))
fig.add_trace(fig_d['data'][0], row=1, col=1)
fig.add_trace(fig_r['data'][0], row=1, col=2)
fig.update_layout(height=800)
fig.show()

In [16]:
px.scatter(latest[latest['Confirmed']>10], x='Confirmed', y='Deaths', color='State/UT', size='Confirmed', 
           text='State/UT', log_x =True, title='Confirmed vs Death')